In [ ]:
# WAV長度處理
def load_trim_and_save_audio(input_file_path, output_file_path, max_length=15.0):
    waveform, sample_rate = torchaudio.load(input_file_path)
    
    # Calculate the number of samples that correspond to max_length seconds
    max_samples = int(max_length * sample_rate)
    
    # Get the total number of samples in the audio file
    num_samples = waveform.size(-1)
    
    if num_samples > max_samples:
        # If the audio is longer than max_length seconds, trim it
        waveform = waveform[:, :max_samples]
        # Save the waveform back to a file
        torchaudio.save(output_file_path, waveform, sample_rate)

In [11]:
def merge_hdf5(files_to_merge, output_path):
    with h5py.File(output_path, 'w') as out_f:
        for file_path in tqdm(files_to_merge):
            with h5py.File(file_path, 'r') as f:
                for group in f:
                    if group in out_f:
                        if 'audio' in f[group] and 'audio' in out_f[group]:
                            data_existing = out_f[group]['audio'][:]
                            data_new = f[group]['audio'][:]
                            merged_data = np.concatenate((data_existing, data_new), axis=0)
                            del out_f[group]['audio']
                            out_f[group].create_dataset('audio', data=merged_data)
                    else:
                        out_f.copy(f[group], group)

In [ ]:
wav_files = glob('MyTTSDataset/train/*wav')
for wav_file in tqdm(wav_files):
    load_trim_and_save_audio(wav_file,wav_file)

In [25]:
from customs.make_custom_dataset import create_dataset
data_dir = "prompts/tet"
create_dataset(data_dir, dataloader_process_only=True)

100%|██████████████████████████████████████| 1.42G/1.42G [00:09<00:00, 160MiB/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 31.74 GiB of which 6.12 MiB is free. Process 12345 has 30.21 GiB memory in use. Including non-PyTorch memory, this process has 1.52 GiB memory in use. Of the allocated memory 603.57 MiB is allocated by PyTorch, and 2.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
os.path.isdir('tet')

False

In [ ]:
data_dir = 'MyTTSDataset/train'
output_dir = data_dir + '_tune'


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

hdf5_files = []
audio_ann_content = ""

# Copy bpe_69.json
bpe_69_path = os.path.join(data_dir, 'bpe_69.json')
if os.path.exists(bpe_69_path):
    shutil.copy(bpe_69_path, os.path.join(output_dir, 'bpe_69.json'))

for root, dirs, files in os.walk(data_dir):
    for file in tqdm(files):
        if file == 'audio_sum.hdf5':
            hdf5_files.append(os.path.join(root, file))
        elif file.endswith('.wav'):
            shutil.copy(os.path.join(root, file), os.path.join(output_dir, file))
        elif file == 'audio_ann_sum.txt':
            with open(os.path.join(root, file), 'r') as f:
                content = f.read()
            audio_ann_content += content + '\n'

with open(os.path.join(output_dir, 'audio_ann_sum.txt'), 'w') as f:
    f.write(audio_ann_content)

merge_hdf5(hdf5_files, os.path.join(output_dir, 'audio_sum.hdf5'))

In [2]:
from tqdm.notebook import tqdm

file_path = '../TTS/JS_Dataset/wavs/audio_ann_sum.txt'

# Step 1: Read all lines from the file
with open(file_path, 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]

write_lines = []
for line in tqdm(lines):
    crop = line.split('|')
    file_name = crop[0]
    text = crop[-1].replace('[ZH]','')
    write_lines.append(f'{file_name}|{text}|{text}')
write_lines = [line if line.endswith('\n') else line + '\n' for line in write_lines]

# Step 3: Write the remaining lines back to the file
with open('metadata.csv', 'w', encoding='utf-8') as f:
    f.writelines(write_lines)

  0%|          | 0/30387 [00:00<?, ?it/s]